In [1]:
import mysql.connector 

mydb = mysql.connector.connect(
    host = "host",
    user = "user",
    passwd = "passwd",
    database = "train_it"
)
mycursor = mydb.cursor()

In [10]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE works_with(truck_id INT, supply_id INT, driver VARCHAR(15), total_sales INT, \
PRIMARY KEY(truck_id, supply_id), FOREIGN KEY(supply_id) REFERENCES supplier(supply_id) ON DELETE CASCADE, FOREIGN KEY(truck_id) REFERENCES truck(truck_id) ON DELETE CASCADE)")

In [15]:
mycursor = mydb.cursor()
mycursor.execute("ALTER TABLE works_with RENAME COLUMN total TO overall")

In [20]:
mycursor = mydb.cursor()
sql = "INSERT INTO works_with(truck_id, supply_id, driver, overall) VALUES (%s, %s, %s, %s)"

val = [
      (102, 4, "Pierre", 1400),
      (101, 3, "Lewis", 1760),
      (107, 2, "Charles", 1870),
      (105, 1, "Valtteri", 1567),
      (100, 4, "Daniel", 1400)
]
mycursor.executemany(sql, val)
mydb.commit()
print(mycursor.rowcount, "record(s) inserted.")

5 record(s) inserted.


In [21]:
mycursor = mydb.cursor()
sql = "INSERT INTO works_with(truck_id, supply_id, driver, overall) VALUES (%s, %s, %s, %s)"
val = (103, 3, "Checo", 1460)
mycursor.execute(sql, val)
mydb.commit()
print(mycursor.rowcount, "record(s) inserted.")

1 record(s) inserted.


In [3]:
mycursor = mydb.cursor()
mycursor.execute ("SELECT * FROM works_with WHERE overall != 1400 AND truck_id = 101")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(101, 3, 'Lewis', 1760)


In [3]:
mycursor = mydb.cursor()
mycursor.execute("SELECT overall FROM works_with LIMIT 5")
myresult = mycursor.fetchone()
for x in myresult:
    print(x)

1400


In [10]:
mycursor = mydb.cursor()
mycursor.execute("SELECT truck_name, lifting_capacity, supply_type, driver FROM truck, supplier, works_with\
WHERE lifting_capacity <> 1.73 GROUP BY lifting_capacity")
myresult = mycursor.fetchone()
for x in myresult:
    print(x)

Scania
9.69
Interior items
Charles


In [1]:
mycursor = mydb.cursor()
sql = "SELECT * FROM works_with WHERE driver = %s ORDER BY truck_id"
drv = ("Daniel",)
mycursor.execute(sql, drv)
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(100, 4, 'Daniel', 1400)


In [3]:
mycursor = mydb.cursor()
sql = "DELETE FROM truck WHERE country = %s"
cntr =  ("China",)
mycursor.execute(sql, cntr)
mydb.commit()
print(mycursor.rowcount, "record(s) deleted.")

1 record(s) deleted.


In [6]:
mycursor = mydb.cursor()
sql = "UPDATE works_with SET driver = %s WHERE truck_id = %s AND overall <> %s"
val = ("Charles", 107, 1760)
mydb.commit()
print(mycursor.rowcount, "record(s) updated.")

-1 record(s) updated.


In [8]:
mycursor = mydb.cursor()
sql = "CREATE TABLE company_supplier (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(21), total VARCHAR(21))"
mycursor.execute(sql)
sql = "INSERT INTO company_supplier (name, total) VALUES (%s, %s)"
val = [
   ("Realm Generation", "2000"),
   ("Blackstone JUS", "1880"),
   ("GoldRush Pearl", "2230")
]
mycursor.executemany(sql,val)
mydb.commit()
print(mycursor.rowcount, "record(s) inserted.")

3 record(s) inserted.


In [4]:
mycursor = mydb.cursor()
sql = "CREATE TABLE orders (id INT AUTO_INCREMENT PRIMARY KEY, brand_id INT, product_id INT)"
mycursor.execute(sql)
sql = "INSERT INTO orders (brand_id, product_id) VALUES (%s, %s)"
val = [
    ("1", "1"),
    ("1", "2"),
    ("2", "3"),
    ("3", "3")
]
mycursor.executemany(sql, val)
mydb.commit()
print(mycursor.rowcount, "record(s) inserted.")

4 record(s) inserted.


In [12]:
sql = "SELECT \
    c.truck_name, o.brand_id\
    FROM truck c\
    JOIN orders o ON c.supply_id = o.brand_id\
    "
mycursor.execute(sql)
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

('Scania', 1)
('DAF', 1)
('Scania', 1)
('DAF', 1)
('MAN', 2)
('IVECO', 2)
('RENAULT', 3)


In [5]:
sql = "SELECT \
     o.product_id, c.truck_name, p.name, p.total\
     FROM orders o\
     JOIN company_supplier p ON o.brand_id = p.id\
     JOIN truck c ON o.product_id = c.supply_id"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(1, 'Scania', 'Realm Generation', '2000')
(1, 'DAF', 'Realm Generation', '2000')
(2, 'MAN', 'Realm Generation', '2000')
(2, 'IVECO', 'Realm Generation', '2000')
(3, 'RENAULT', 'Blackstone JUS', '1880')
(3, 'RENAULT', 'GoldRush Pearl', '2230')
